# MOX CREDIT SCORING PROJECT

# ETL PROCESS + TARGET MAKING

### Importing the necessary libraries

In [48]:
import os
import re
import json
import numpy as np
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

### URI conection with mongodb

In [ ]:
mongo_uri = os.environ['URL_DATABASE']

### Making a client in MongoClient

In [50]:
client = MongoClient(mongo_uri)

### Access to the 'imms' database

In [51]:
db = client['validationsdb']

### Access collection 'imms_example'

In [52]:
collection = db['validations']

### Extract all records from the collection

In [53]:
mox_final = collection.find({})


### Create a Pandas DataFrame with the extracted data

In [54]:
df_mox = pd.DataFrame(list(mox_final))

### Observing the first 5 data

In [55]:
df_mox.head()

,_id,transactionId,createdAt,curp,dob,semanasCotizadas,semanasDescontadas,semanasReintegradas,historialLaboral
0,64ac8c8c6a450cc3b38ff1fe,c81b9a81-60ce-4134-87c0-b667574b5859,"Tue, 04 Apr 2023 17:53:49 GMT",efe93735d0990e9b6f2d0428e8b997ddf13642190476e4...,981119,44,0,0,"[{'fechaBaja': 'Vigente', 'fechaAlta': '06/01/..."
1,64ac8c8c6a450cc3b38ff1ff,71333ea4-62af-4a9e-b01d-42fff40ca033,"Tue, 04 Apr 2023 18:28:11 GMT",49d09e0ee642ad942a03a308dbf459ce729d934efbf21a...,971113,32,0,0,"[{'fechaBaja': 'Vigente', 'fechaAlta': '10/20/..."
2,64ac8c8c6a450cc3b38ff200,2e8e0c56-f108-4e07-815e-71a9ce55e363,"Tue, 04 Apr 2023 18:28:11 GMT",efe93735d0990e9b6f2d0428e8b997ddf13642190476e4...,981119,44,0,0,"[{'fechaBaja': 'Vigente', 'fechaAlta': '06/01/..."
3,64ac8c8c6a450cc3b38ff201,6e98c639-d0b0-4590-bc03-b57fa2f178b2,"Tue, 04 Apr 2023 18:35:13 GMT",a832a2f4b357a6088106c7befe9da78440b98f9ecc6428...,700124,1205,0,0,"[{'fechaBaja': 'Vigente', 'fechaAlta': '08/25/..."
4,64ac8c8c6a450cc3b38ff202,c216f5a3-b180-47d2-8376-0c213aa0e3ba,"Tue, 04 Apr 2023 18:35:13 GMT",bf5fb50e1475d6fc2a4a9dfd31167b5399759f7c8606e4...,650906,618,78,0,"[{'fechaBaja': 'Vigente', 'fechaAlta': '12/08/..."


In [56]:
df_mox.shape

(27470, 9)

### Looking duplicated values

In [57]:
df_mox['curp'].value_counts().head(50)

curp
d205bdb41d60595fd3862b681b8d8f4cd22769b54c62ea1e89bacc3d934ce207    77
be8abe02cc8e1e9322258c07f3b0d8d249601a868efa81bbd05ce75a0676dff1    43
0033fb7021ea8e8ed69f69a585314c0a3f579e65ef0f9f3191d315427248afa5    30
c3e27a5eb9ced3c643fda8fae7f9c86d055554026bd531658712260860e8ebe1    22
0e4b73e2054aee05406262724f4239dd13fe23a17603ac919d418a927e74a910    19
c1dcc75d9497f79bbd908916a31d66b6db7c6576e9cc078a2796ed95cfb9a36c    17
52618826094047150986fadb60a4b737458e8463b13d737489ba07e201cee3d4    16
d3c5020ac36d1213c7a5e75cbcdbff46e51d3bcbdaec12f813d31c9d6d0e46e7    12
bf5fb50e1475d6fc2a4a9dfd31167b5399759f7c8606e4a77cb786b475b8d604    11
e048f224c4a1468c81ce627b973018492f7ebd8ddcb65b14edaefe9a2a0874d4    10
641365e3df64b0fcd3cd4c2646f42151834e0ce278fbfa0011d9266b0dedcaf3     9
c3aa9697142fff55b66d4902c5609cc306f6ac6c9a1ed399443cca9ce18728ab     8
4554f5132d172971dfb0c0ce3c53068c87b32a227733478966401055f6d3b50a     7
c851c0727b38b3a8ea4c53422901f2e3f50223786c17dc179e5f0783bfbccdc3     7
d

### fixing the createdAt column

In [58]:
def fix_dates(dates):
    """Parses date/time strings in two different formats and returns a datetime object or a formatted string.

    Args:
        dates (str): A string representing a date and time
    Returns:
        dates (datetime): the date in a fixed format

    try:
        return pd.to_datetime(dates, format='%a, %d %b %Y %H:%M:%S %Z')
    except ValueError:
        return pd.to_datetime(dates[:-5], format='%Y-%m-%dT%H:%M:%S')
    """

    try:
        dates = pd.to_datetime(dates,format='%a, %d %b %Y %H:%M:%S %Z').strftime('%Y-%m-%d %H:%M:%S')
        formato_str = '%Y-%m-%d %H:%M:%S'
        dates = datetime.strptime(dates, formato_str)
        return dates
    except ValueError:
        return pd.to_datetime(dates[:-5], format='%Y-%m-%dT%H:%M:%S')

In [59]:
df_mox['createdAt'] = df_mox['createdAt'].apply(fix_dates)

### Sorting the data by most current dates

In [60]:
df_mox = df_mox.sort_values('createdAt', ascending=False)

### Deleting duplicated curps

In [61]:
df_mox = df_mox.drop_duplicates('curp')

### Reseting indexes in the table

In [62]:
df_mox.reset_index(drop= True, inplace= True)

### Deleting unused columns

In [63]:
df_mox = df_mox.drop(['_id', 'transactionId','semanasDescontadas','semanasReintegradas'], axis= 1)

### Dividing the data into 2 different dataframes
### first with current_place and second without it

In [64]:
df_con_entidad = pd.DataFrame(columns=df_mox.columns)
df_sin_entidad = pd.DataFrame(columns=df_mox.columns)

for index, row in df_mox.iterrows():
    entidad_presente = False
    for d in row['historialLaboral']:
        if 'entidadFederativa' in d:
            entidad_presente = True
            break

    if entidad_presente:
        df_con_entidad = pd.concat([df_con_entidad, pd.DataFrame([row])], ignore_index=True)
    else:
        df_sin_entidad = pd.concat([df_sin_entidad, pd.DataFrame([row])], ignore_index=True)

### Unnesting the df_con_entidad

In [65]:
output = []

for index, list_element in enumerate(df_con_entidad['historialLaboral']):
    person_data = {
        'fechaAlta': [],
        'fechaBaja': [],
        'salarioBaseCotizacion': [],
        'nombrePatron': [],
        'entidadFederativa': []
    }

    for d in list_element:
        person_data['fechaAlta'].append(d.get('fechaAlta'))
        person_data['fechaBaja'].append(d.get('fechaBaja'))
        person_data['salarioBaseCotizacion'].append(d.get('salarioBaseCotizacion'))
        person_data['nombrePatron'].append(d.get('nombrePatron'))
        person_data['entidadFederativa'].append(d.get('entidadFederativa'))

    output.append(person_data)

output_df = pd.DataFrame(output)

df_con_entidad = pd.concat([df_con_entidad, output_df], axis=1)

### Unnesting the df_sin_entidad

In [66]:
output = []

for index, list_element in enumerate(df_sin_entidad['historialLaboral']):
    person_data = {
        'fechaAlta': [],
        'fechaBaja': [],
        'salarioBaseCotizacion': [],
        'nombrePatron': []
    }

    for d in list_element:
        person_data['fechaAlta'].append(d.get('fechaAlta'))
        person_data['fechaBaja'].append(d.get('fechaBaja'))
        person_data['salarioBaseCotizacion'].append(d.get('salarioBaseCotizacion'))
        person_data['nombrePatron'].append(d.get('nombrePatron'))

    output.append(person_data)

output_df = pd.DataFrame(output)

df_sin_entidad.reset_index(drop=True,inplace=True)

df_sin_entidad = pd.concat([df_sin_entidad, output_df], axis=1)

### Joinin the 2 dataframes into a single dataframe

In [67]:
if len(df_sin_entidad.columns) < len(df_con_entidad.columns):
    df_sin_entidad = df_sin_entidad.reindex(columns=df_con_entidad.columns)

df_final = pd.concat([df_sin_entidad, df_con_entidad], ignore_index=True)

### Eliminating the nested columns, since we extracted the information from it

In [68]:
df_final.drop(['historialLaboral','createdAt'], axis=1, inplace=True)

### Calculating the current date

In [69]:
now = datetime.now()
current_year = now.year

### Creating the age column

In [70]:
df_final['edad'] = 0

### Creating a function that calculates the age of the employee

In [71]:
def calculate_age(dob, now):
    """
    Calculates the age from the CURP of a person and the current date.

    Args:
        curp (str): The employee's CURP in YY/MM/DD format.
        now (datetime): The current date.

    Returns:
        int: the age of the employee in years.
    """

    digits_year_curp = re.findall(r'\d+', dob)
    year_of_birth = int(digits_year_curp[0][:2])
    month_of_birth = int(digits_year_curp[0][2:4])
    day_of_birth = int(digits_year_curp[0][4:6])
    if year_of_birth < current_year - 2000:
        birthdate = datetime(year_of_birth + 2000, month_of_birth, day_of_birth)
        age = now.year - birthdate.year - ((now.month, now.day) < (birthdate.month, birthdate.day))
    elif year_of_birth > current_year - 2000:
        birthdate = datetime(year_of_birth + 1900, month_of_birth, day_of_birth)
        age = now.year - birthdate.year - ((now.month, now.day) < (birthdate.month, birthdate.day))

    if age < 0:
        raise ValueError("La fecha de nacimiento es posterior a la fecha actual.")

    return age

### Applying the calculate_age function

In [72]:
df_final['edad'] = df_final['dob'].apply(lambda x: calculate_age(x, now))

### Setting a constant

In [73]:
VAR_CURRENT = 'vigente'

### Creating a function that calculates job seniority

In [74]:
def calculate_employment_seniority(fila):
    """
    Calculates the length of time an employee has been employed in their current job in months.

    Args:
        fila (pandas.DataFrame): A DataFrame containing the employee's job history.

    Returns:
        int: The number of months the employee has been employed in their current job.
    """


    current_index = [i for i, x in enumerate(fila['fechaBaja'])
                    if VAR_CURRENT in x.lower().replace(' ', '')]

    if len(current_index) > 0:
        last_current = current_index[-1]
        date = datetime.strptime(fila['fechaAlta'][last_current], '%m/%d/%Y')
        current_date = datetime.now()
        difference_dates = relativedelta(current_date, date)
        age_months = difference_dates.years * 12 + difference_dates.months
        return age_months

    else:
        return 0

### Applying the calculate_employment_seniority function

In [75]:
df_final['antiguedad_laboral_meses'] = df_final.apply(calculate_employment_seniority, axis=1)

### Creating a function that calculates the unemployment time

In [76]:
def unemployment_time(table):
    """
    Calculates the length of time someone has been unemployed in months.

    Args:
        table (pandas.DataFrame): A DataFrame containing the employee's job history.

    Returns:
        int: The number of months of that person has been unemployed.
    """
    AUX = 0

    if len(table['fechaBaja']) > 0:
        if any(VAR_CURRENT in x.lower().replace(' ', '') for x in table['fechaBaja']):
            return 0
        else:
            for fecha in table['fechaBaja']:
                try:
                    date = datetime.strptime(fecha, '%m/%d/%Y')
                    break
                except ValueError:
                    pass
            else:
                return 0

            current_date = datetime.now()
            difference_dates = relativedelta(current_date, date)
            time_months = difference_dates.years * 12 + difference_dates.months
            return time_months
    else:
        return 0

### Applying the unemployment_time function

In [77]:
df_final['tiempo_desempleado'] = df_final.apply(unemployment_time, axis=1)

### Creating the monthly salary column

In [78]:
def get_monthly_salary(lista):
    if len(lista) > 0:
        salario_mensual = [round(float(valor.replace('$', '').replace(',', '')) * 30.4165 ,2) for valor in lista]
        return salario_mensual
    else:
        return []

### Applying calculate_income function

In [79]:
df_final['salarioMensual'] = df_final['salarioBaseCotizacion'].apply(get_monthly_salary)

### Setting the income column

In [80]:
df_final['ingreso'] = 0

### Creating a function that calculates the current income

In [81]:
def calculate_income(low_date, monthly_salary):
    """
    Calculates a person's current income based on their monthly salary.

    Args:
        low_date (list): A list of dates indicating when the monthly salary was last updated.
        monthly_salary (list): A list of monthly salaries.

    Returns:
        float: The person's current income (or 0 if the salary information is not current).
    """

    if len(low_date) > 0 :
        for i in range(len(low_date)):
            if VAR_CURRENT in low_date[i].lower():
                return float(monthly_salary[i])
        else:
            return 0
    else:
        return 0




### Applying the calculate_income function

In [82]:
df_final['ingreso'] = df_final.apply(lambda x: calculate_income(x['fechaBaja'],
                                                                x['salarioMensual']),
                                    axis=1)

In [83]:
df_final

,curp,dob,semanasCotizadas,fechaAlta,fechaBaja,salarioBaseCotizacion,nombrePatron,entidadFederativa,edad,antiguedad_laboral_meses,tiempo_desempleado,salarioMensual,ingreso
0,015c5728cede59dd3be1c946a576420d64fcf195a31b3d...,820504,0,[],[],[],[],NaN,41,0,0,[],0.00
1,139213ea34080bbd7886df37978cc0e165b2a0843ec397...,961204,0,[],[],[],[],NaN,26,0,0,[],0.00
2,93b274ef0ad44adb14e5a792d53c89c5a3d1bfd29d6215...,830115,0,[],[],[],[],NaN,40,0,0,[],0.00
3,f11a33818c5519df12b0b3c1662a805f3587375ce84db8...,700823,0,[],[],[],[],NaN,52,0,0,[],0.00
4,0c3c4215a70e780e6a6e1b4de18934c9da6f128a4bd3ae...,050515,0,[],[],[],[],NaN,18,0,0,[],0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26466,b54e6acec49946f852066e14990ae81c7aafc6e8ea0429...,720319,60,[04/06/2022],[Vigente],[$365.92],[COPLASCORP],[CHIHUAHUA],51,15,0,[11130.01],11130.01
26467,6ed5ca37cf961f87c6592d6bb8dbcb5264317a6b22bad5...,851223,27,"[01/08/2021, 10/23/2019, 09/26/2017, 04/01/200...","[02/02/2021, 12/18/2019, 11/09/2017, 04/19/200...","[$150, $195.53, $290.82, $40.03, $41.91, $40.0...","[AEME ASESORES EN MERCADOTECNIA, SERVICIOS OPE...","[COAHUILA DE ZARAGOZA, COAHUILA DE ZARAGOZA, C...",37,0,29,"[4562.47, 5947.34, 8845.73, 1217.57, 1274.76, ...",0.00
26468,31ed11cedba035077fc99748f298a60f252d60d7c51f5c...,870830,572,"[08/24/2022, 08/16/2022, 08/01/2022, 02/08/202...","[12/26/2022, 09/16/2022, 08/15/2022, 04/07/202...","[$443.05, $190.23, $190.23, $180.68, $563.5, $...","[CORPORATIVO BORO, BRAVO SPIRITS, BRAVO SPIRIT...","[JALISCO, MÉXICO, HIDALGO, MÉXICO, DISTRITO FE...",35,0,6,"[13476.03, 5786.13, 5786.13, 5495.65, 17139.7,...",0.00
26469,5047618e4107aad5ede8573aaec99881eef896747aaf42...,790619,1097,"[12/16/2022, 12/12/2022, 11/01/2022, 07/16/202...","[Vigente, 12/15/2022, 12/08/2022, 09/06/2022, ...","[$1294.15, $838.36, $530.85, $1437.08, $704.82...","[INX INTERNATIONAL DE MEXICO, SAVARIA MEXICO, ...","[QUERÉTARO, QUERÉTARO, DISTRITO FEDERAL, NUEVO...",44,7,0,"[39363.51, 25499.98, 16146.6, 43710.94, 21438....",39363.51


### Establishing constants to calculate retirement

In [84]:
RETIREMENT_AGE = 65
RETIREMENT_WEEKS = 1500

In [85]:
df_final['semanasCotizadas'] = df_final['semanasCotizadas'].astype(int)

### Creating a function that informs if someone is retired or not

In [86]:
def calculate_retirement(age, listed_weeks, income):
    """
    Determines whether a person is eligible for a pension based on their age and number of weeks worked.

    Args:
        age (int): The person's age in years.
        listed_weeks (int): The number of weeks the person has worked.

    Returns:
        int: 1 if the person is eligible for a pension, 0 otherwise.
    """
    if income == 0:
        return int(age >= RETIREMENT_AGE and listed_weeks >= RETIREMENT_WEEKS)
    else:
        return 0

### Applying the calculate_pensioner function

In [87]:
df_final['jubilado'] = df_final.apply(lambda row: calculate_retirement(int(row['edad']),
                                                                    int(row['semanasCotizadas']),
                                                                    int(row['ingreso'])),
                                    axis=1)

In [88]:
def time_worked(start_dates, end_dates):
    """
    Calculate the difference in months between two lists of dates in the format "month/year".

    Args:
        start_dates (list): A list of start dates in the format "month/year".
        end_dates (list): A list of end dates in the format "month/year".

    Returns:
        list: A list of integers representing the difference in months between each pair of start and end dates.
    """
    months_list = []
    for start, end in zip(start_dates, end_dates):
        try:
            start_date = datetime.strptime(start, '%m/%d/%Y')
            end_date = datetime.strptime(end, '%m/%d/%Y')
        except ValueError:
            try:
                start_date = datetime.strptime(start, '%d/%m/%Y')
                end_date = datetime.strptime(end, '%d/%m/%Y')
            except ValueError:
                try:
                    start_date = datetime.strptime(start, '%Y-%m-%d')
                    end_date = datetime.strptime(end, '%Y-%m-%d')
                except ValueError:
                    start_date = datetime.strptime(start, '%m-%d-%Y')
                    end_date = datetime.strptime(end, '%m-%d-%Y')
        months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
        months_list.append(months)

    return months_list

In [89]:
df_final['tiempo_trabajado'] = df_final.apply(lambda row: time_worked(row['fechaAlta'],
                                                                    row['fechaBaja'])
                                            if row['jubilado'] == 1 else [], axis=1)

In [90]:
def retired_wages(departure_date, monthly_salary):
    """
    Returns a list of the monthly salaries that a person has earned in the last 3 years.

    Args:
        admission_date (list): A list of dates indicating when the person started each job.
        monthly_salary (list): A list of monthly salaries earned by the person.

    Returns:
        list: A list of monthly salaries earned by the retired person in the last 10 years.
    """

    wages = []
    if departure_date:
        min_year = max([int(date.split('/')[-1]) for date in departure_date]) - 10
    else:
        min_year = current_year - 10

    for date, salary in zip(departure_date, monthly_salary):
        anio = int(date.split('/')[-1])
        if anio >= min_year:
            wages.append(salary)

    return wages

In [91]:
df_final['salarios_jubilacion'] = df_final.apply(lambda row: retired_wages(row['fechaBaja'],
                                                                        row['salarioMensual'])
                                                if row['jubilado'] == 1 else [], axis=1)

### Creating a function that calculates the income of a retired person

In [92]:
def retired_income(salaries):
    """
    Calculates the average salary of retired individuals in a given DataFrame.

    Args:
        salaries (list): The name of the column containing the lists of salaries.

    Returns:
        float: The average salary of retired individuals in the specified column.
    """

    try:
        return round(sum(salaries) / len(salaries), 2)
    except ZeroDivisionError:
        return 0

### Applying the retired_income function

In [93]:
df_final['ingreso'] = df_final.apply(lambda row: retired_income(row['salarios_jubilacion'])
                                    if row['jubilado'] == 1 else row['ingreso'], axis=1)

### Creating a constant

In [94]:
YEARS_TO_VERIFY = 5

### Calculating the current year minus the previous constant

In [95]:

current_year_less_5 = current_year - YEARS_TO_VERIFY

### Creating function that calculates the number of jobs in the last 5 years

In [96]:
def jobs_last_5_years(dates_in, dates_out, seniority, employer_names):
    """
    Calculates the number of jobs a person has had
    in the last 5 years based on the dates they ended,
    considering the employer names.

    Args:
        dates_in (list): A list of dates indicating when the person's jobs started.
        dates_out (list): A list of dates indicating when the person's jobs ended.
        seniority (int): The person's seniority (months of experience).
        employer_names (list): A list of employer names.

    Returns:
        int: The number of jobs the person had in the last 5 years.
    """
    last_five_years = datetime.now() - timedelta(days=5 * 365)
    num_jobs = 0
    previous_employer = None

    for i in range(len(dates_out)):
        if VAR_CURRENT in dates_out[i].lower():
            date = datetime.strptime(dates_in[i], '%m/%d/%Y')
        else:
            date = datetime.strptime(dates_out[i], '%m/%d/%Y')

        if last_five_years <= date <= datetime.now():
            if i == 0 or employer_names[i] != previous_employer:
                num_jobs += 1
            previous_employer = employer_names[i]

    if num_jobs == 0 and seniority > 0:
        num_jobs += 1

    return num_jobs

### Applying the jobs_last_5_years function

In [97]:
df_final['trabajos_ultimos_5'] = df_final.apply(lambda row: jobs_last_5_years(row['fechaAlta'],
                                                row['fechaBaja'],row['antiguedad_laboral_meses'],
                                                row['nombrePatron']),
                                                axis=1)

### Creating function that calculates salaries in the last 3 years

In [98]:
def wages_3_last_years(admission_date, monthly_salary):
    """
    Returns a list of the monthly salaries that a person has earned in the last 3 years.

    Args:
        admission_date (list): A list of dates indicating when the person started each job.
        monthly_salary (list): A list of monthly salaries earned by the person.

    Returns:
        list: A list of monthly salaries earned by the person in the last 3 years.
    """

    wages = []

    for date, salary in zip(admission_date, monthly_salary):
        anio = int(date.split('/')[-1])
        if anio >= current_year - 3:
            wages.append(salary)

    return wages


### Applying the wages_3_last_years function

In [99]:
df_final['salarios_ultimos_3_anios'] = df_final.apply(lambda row:
                                                    wages_3_last_years(row['fechaAlta'],
                                                                        row['salarioMensual']),
                                                    axis=1)


### Creating a function that calculates income growth

In [100]:
def income_growth(wages , dates):
    """
    In this function, the income of the last 3 years is read
    and the percentage growth of the same is returned.

    Args:
        wages (list): salaries in the last 3 years

    Returns:
        float: describes the growth income in the last 3 years
    """

    if len(dates) > 0 and VAR_CURRENT in dates[0].lower():
        numerical_salaries = []
        for salary in wages:

            if isinstance(salary, (int, float)):
                numerical_salaries.append(salary)

            elif isinstance(salary, str):
                salary_without_dollar = salary.replace('$', '')

                try:
                    salary_numeric = float(salary_without_dollar)
                    numerical_salaries.append(salary_numeric)

                except ValueError as er:
                    print(er)

        if len(numerical_salaries) >= 2:
            start_average_salary = numerical_salaries[-1]
            final_average_salary = numerical_salaries[0]
            growth = (final_average_salary - start_average_salary) / start_average_salary * 100

        else:
            growth = 0

        return round(growth,2)

    else:
        growth = 0
        return round(growth,2)

### Applying the income_growth function

In [101]:
df_final['crecimiento_ingreso']=df_final.apply(lambda x:income_growth(x['salarios_ultimos_3_anios'],
                                                                        x['fechaBaja']), axis=1)

### Calculating the current location of everyone in the dataframe

In [102]:
def current_place(place, dates):
    """
    Using the work history we set the current State where the person lives

    Args:
        place (list): States where the employee has worked
        dates (list): Dates to verify if the employee is working or not

    Returns:
        String: Current place of the person
    """
    if len(dates) > 0:
        for i in range(len(dates)):
            if VAR_CURRENT in dates[i].lower():
                if isinstance(place, list):
                    return place[i]
                else:
                    return place
        if isinstance(place, list):
            return place[0]
        else:
            return None
    else:
        return None

### Applying the current_place function

In [103]:
df_final['lugar_actual'] = df_final.apply(lambda row:current_place(row['entidadFederativa'],
                                                                    row['fechaBaja']), axis=1)

In [104]:
df_final

,curp,dob,semanasCotizadas,fechaAlta,fechaBaja,salarioBaseCotizacion,nombrePatron,entidadFederativa,edad,antiguedad_laboral_meses,tiempo_desempleado,salarioMensual,ingreso,jubilado,tiempo_trabajado,salarios_jubilacion,trabajos_ultimos_5,salarios_ultimos_3_anios,crecimiento_ingreso,lugar_actual
0,015c5728cede59dd3be1c946a576420d64fcf195a31b3d...,820504,0,[],[],[],[],NaN,41,0,0,[],0.00,0,[],[],0,[],0.00,None
1,139213ea34080bbd7886df37978cc0e165b2a0843ec397...,961204,0,[],[],[],[],NaN,26,0,0,[],0.00,0,[],[],0,[],0.00,None
2,93b274ef0ad44adb14e5a792d53c89c5a3d1bfd29d6215...,830115,0,[],[],[],[],NaN,40,0,0,[],0.00,0,[],[],0,[],0.00,None
3,f11a33818c5519df12b0b3c1662a805f3587375ce84db8...,700823,0,[],[],[],[],NaN,52,0,0,[],0.00,0,[],[],0,[],0.00,None
4,0c3c4215a70e780e6a6e1b4de18934c9da6f128a4bd3ae...,050515,0,[],[],[],[],NaN,18,0,0,[],0.00,0,[],[],0,[],0.00,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26466,b54e6acec49946f852066e14990ae81c7aafc6e8ea0429...,720319,60,[04/06/2022],[Vigente],[$365.92],[COPLASCORP],[CHIHUAHUA],51,15,0,[11130.01],11130.01,0,[],[],1,[11130.01],0.00,CHIHUAHUA
26467,6ed5ca37cf961f87c6592d6bb8dbcb5264317a6b22bad5...,851223,27,"[01/08/2021, 10/23/2019, 09/26/2017, 04/01/200...","[02/02/2021, 12/18/2019, 11/09/2017, 04/19/200...","[$150, $195.53, $290.82, $40.03, $41.91, $40.0...","[AEME ASESORES EN MERCADOTECNIA, SERVICIOS OPE...","[COAHUILA DE ZARAGOZA, COAHUILA DE ZARAGOZA, C...",37,0,29,"[4562.47, 5947.34, 8845.73, 1217.57, 1274.76, ...",0.00,0,[],[],2,[4562.47],0.00,COAHUILA DE ZARAGOZA
26468,31ed11cedba035077fc99748f298a60f252d60d7c51f5c...,870830,572,"[08/24/2022, 08/16/2022, 08/01/2022, 02/08/202...","[12/26/2022, 09/16/2022, 08/15/2022, 04/07/202...","[$443.05, $190.23, $190.23, $180.68, $563.5, $...","[CORPORATIVO BORO, BRAVO SPIRITS, BRAVO SPIRIT...","[JALISCO, MÉXICO, HIDALGO, MÉXICO, DISTRITO FE...",35,0,6,"[13476.03, 5786.13, 5786.13, 5495.65, 17139.7,...",0.00,0,[],[],5,"[13476.03, 5786.13, 5786.13, 5495.65, 17139.7]",0.00,JALISCO
26469,5047618e4107aad5ede8573aaec99881eef896747aaf42...,790619,1097,"[12/16/2022, 12/12/2022, 11/01/2022, 07/16/202...","[Vigente, 12/15/2022, 12/08/2022, 09/06/2022, ...","[$1294.15, $838.36, $530.85, $1437.08, $704.82...","[INX INTERNATIONAL DE MEXICO, SAVARIA MEXICO, ...","[QUERÉTARO, QUERÉTARO, DISTRITO FEDERAL, NUEVO...",44,7,0,"[39363.51, 25499.98, 16146.6, 43710.94, 21438....",39363.51,0,[],[],5,"[39363.51, 25499.98, 16146.6, 43710.94, 21438.16]",83.61,QUERÉTARO


### Creating a function that sets the name of the current job

In [105]:
def current_job(low_date, job_places):
    """
    Calculates a person's current job name.

    Args:
        low_date (list): A list of dates indicating when the monthly salary was last updated.
        job_places (list): A list of jobs names.

    Returns:
        str: The person's current jot (or None if the job information is not current).
    """

    if len(low_date) > 0 :
        for i in range(len(low_date)):
            if VAR_CURRENT in low_date[i].lower():
                return job_places[i]
        else:
            return None
    else:
        return None

### Applying the current_job function

In [106]:
df_final['patronActual'] = df_final.apply(lambda x: current_job(x['fechaBaja'],
                                                                x['nombrePatron']),
                                    axis=1)

In [107]:
df_scoring = df_final[['ingreso','antiguedad_laboral_meses','tiempo_desempleado',
                    'trabajos_ultimos_5','semanasCotizadas','edad','crecimiento_ingreso',
                    'lugar_actual','patronActual', 'jubilado']]

# Gathering data from external sources

In [108]:
tabla_ENIGH = pd.read_csv('../dataset/enigh.csv')
ITAEE_GRAL = pd.read_csv('../dataset/itaee_gral_2023.csv')

### Calculating the current place

In [109]:
df_scoring['lugar_actual'] = df_scoring['lugar_actual'].replace('DISTRITO FEDERAL', 'CIUDAD DE MÉXICO')

C:\Users\DauntLess\AppData\Local\Temp\ipykernel_12652\3191708592.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scoring['lugar_actual'] = df_scoring['lugar_actual'].replace('DISTRITO FEDERAL', 'CIUDAD DE MÉXICO')


### Creating a function that matchs the ENIGH with our table

In [110]:
def merge_data_enigh(df_scoring, tabla_ENIGH):
    for index, row in df_scoring.iterrows():
        lugar_actual = row['lugar_actual']
        matching_row = tabla_ENIGH[tabla_ENIGH['Entidades'] == lugar_actual]
        if not matching_row.empty:
            df_scoring.loc[index, 'liquidez_lugar_actual'] = matching_row['Liquidez'].values[0]
            df_scoring.loc[index, 'gasto_lugar_actual'] = matching_row['Gasto Total'].values[0]
            decil_ingreso = np.digitize(row['ingreso'],
                                        bins=matching_row.loc[:, '1':'10'].values[0], right=False)
            df_scoring.loc[index, 'decil_ingreso_ENIGH'] = decil_ingreso

### Applying the merge_data_enigh function

In [111]:
merge_data_enigh(df_scoring, tabla_ENIGH)

C:\Users\DauntLess\AppData\Local\Temp\ipykernel_12652\2633732614.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scoring.loc[index, 'liquidez_lugar_actual'] = matching_row['Liquidez'].values[0]
C:\Users\DauntLess\AppData\Local\Temp\ipykernel_12652\2633732614.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scoring.loc[index, 'gasto_lugar_actual'] = matching_row['Gasto Total'].values[0]
C:\Users\DauntLess\AppData\Local\Temp\ipykernel_12652\2633732614.py:10: SettingWithCopyWarning: 
A value is tr

In [112]:
df_scoring

,ingreso,antiguedad_laboral_meses,tiempo_desempleado,trabajos_ultimos_5,semanasCotizadas,edad,crecimiento_ingreso,lugar_actual,patronActual,jubilado,liquidez_lugar_actual,gasto_lugar_actual,decil_ingreso_ENIGH
0,0.00,0,0,0,0,41,0.00,None,None,0,NaN,NaN,NaN
1,0.00,0,0,0,0,26,0.00,None,None,0,NaN,NaN,NaN
2,0.00,0,0,0,0,40,0.00,None,None,0,NaN,NaN,NaN
3,0.00,0,0,0,0,52,0.00,None,None,0,NaN,NaN,NaN
4,0.00,0,0,0,0,18,0.00,None,None,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26466,11130.01,15,0,1,60,51,0.00,CHIHUAHUA,COPLASCORP,0,15506.0,10954.0,5.0
26467,0.00,0,29,2,27,37,0.00,COAHUILA DE ZARAGOZA,None,0,15043.0,12717.0,0.0
26468,0.00,0,6,5,572,35,0.00,JALISCO,None,0,14392.0,13852.0,0.0
26469,39363.51,7,0,5,1097,44,83.61,QUERÉTARO,INX INTERNATIONAL DE MEXICO,0,15321.0,14301.0,9.0


### Creating a function that returns porcentual liquidity of that employee

In [113]:
def calculate_percentage_liquidity(row):
    return ((row['ingreso'] - row['liquidez_lugar_actual']) / row['liquidez_lugar_actual']) * 100

### Applying the calculate_percentage_liquidity function

In [114]:
df_scoring['liquidez_porcentual'] = df_scoring.apply(calculate_percentage_liquidity, axis=1)

C:\Users\DauntLess\AppData\Local\Temp\ipykernel_12652\3352197647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scoring['liquidez_porcentual'] = df_scoring.apply(calculate_percentage_liquidity, axis=1)


### Making a function that calculates the cost of living

In [115]:
def cost_of_living(row):
    return ((row['ingreso'] - row['gasto_lugar_actual']) / row['gasto_lugar_actual']) * 100

### Applying the cost_of_living function

In [116]:
df_scoring['costo_de_vida'] = df_scoring.apply(cost_of_living, axis=1)

C:\Users\DauntLess\AppData\Local\Temp\ipykernel_12652\768268384.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scoring['costo_de_vida'] = df_scoring.apply(cost_of_living, axis=1)


In [117]:
df_scoring = df_scoring.drop(['liquidez_lugar_actual', 'gasto_lugar_actual'] , axis= 1)

### Creating a function that matchs the ITAEE with our table

In [118]:
def merge_data_itaee(df_scoring, ITAEE_GREAL):
    if 'crecimiento_gral' not in df_scoring.columns:
        df_scoring['crecimiento_gral'] = None
    for index, row in df_scoring.iterrows():
        try:
            lugar_actual = row['lugar_actual'].lower()
            matching_row = ITAEE_GREAL[ITAEE_GREAL['entidad_federativa'].str.lower() == lugar_actual]
            if not matching_row.empty:
                df_scoring.loc[index, 'crecimiento_gral'] = matching_row['2023|Anual'].values[0]
        except AttributeError:
            df_scoring.loc[index, 'crecimiento_gral'] = None

### Applying the merge_data_itaee function

In [119]:
merge_data_itaee(df_scoring, ITAEE_GRAL)

### Deleting the first 90 rows , trash data

In [120]:
df_scoring = df_scoring.drop(range(90))
df_scoring = df_scoring.reset_index(drop=True)

### Deleting duplicated values

In [121]:
df_scoring.duplicated().sum()

42

In [122]:
df_scoring = df_scoring.drop_duplicates()
df_scoring = df_scoring.reset_index(drop=True)

In [123]:
df_scoring.duplicated().sum()

0

### Fixing NaN values

In [124]:
df_scoring['decil_ingreso_ENIGH'] = df_scoring['decil_ingreso_ENIGH'].fillna(6)
df_scoring['liquidez_porcentual'] = df_scoring['liquidez_porcentual'].fillna(0)
df_scoring['costo_de_vida'] = df_scoring['costo_de_vida'].fillna(0)
df_scoring['crecimiento_gral'] = df_scoring['crecimiento_gral'].fillna(0)
df_scoring['lugar_actual'] = df_scoring['lugar_actual'].fillna('Sin dato')
df_scoring['patronActual'] = df_scoring['patronActual'].fillna('Sin dato')

### Creating the target column

In [250]:
df_scoring['target'] = 0

### Making a function that calculates a target value
### using business rules as conditionals

In [ ]:
def creating_target(row):
    edad = row['edad']
    ingreso = row['ingreso']
    semanas = row['semanasCotizadas']
    antiguedad = row['antiguedad_laboral_meses']
    tiempo_desempleado = row['tiempo_desempleado']
    trabajos = row['trabajos_ultimos_5']
    crecimiento = row['crecimiento_ingreso']
    decil = row['decil_ingreso_ENIGH']
    liquidez = row['liquidez_porcentual']
    costo = row['costo_de_vida']
    jubilado = row['jubilado']
    region = row['crecimiento_gral']

    with open('../business_rules/business_rules.json', 'r', encoding='utf-8') as file:
        rules_file = json.load(file)

    if rules_file['edad']['limites_08']['min'] <= edad <= rules_file['edad']['limites_08']['max']:
        row['target'] += rules_file['edad']['value_08']
    elif (rules_file['edad']['limites_06']['min1'] <= edad < rules_file['edad']['limites_06']['max1']):
        row['target'] += rules_file['edad']['value_06']
    elif (rules_file['edad']['limites_06']['min2'] < edad <= rules_file['edad']['limites_06']['max2']):
        row['target'] += rules_file['edad']['value_06']
    elif (rules_file['edad']['limites_05']['min1'] <= edad < rules_file['edad']['limites_05']['max1']):
        row['target'] += rules_file['edad']['value_05']
    elif (rules_file['edad']['limites_05']['min2'] < edad <= rules_file['edad']['limites_05']['max2']):
        row['target'] += rules_file['edad']['value_05']
    elif (rules_file['edad']['limites_03']['min1'] <= edad < rules_file['edad']['limites_03']['max1']):
        row['target'] += rules_file['edad']['value_03']
    elif (rules_file['edad']['limites_03']['min2'] < edad <= rules_file['edad']['limites_03']['max2']):
        row['target'] += rules_file['edad']['value_03']
    else:
        row['target'] += rules_file['edad']['value_00']

    if ingreso >= rules_file['ingreso']['limites_10']['min']:
        row['target'] += rules_file['ingreso']['value_10']
    elif rules_file['ingreso']['limites_09']['min'] <= ingreso < rules_file['ingreso']['limites_09']['max']:
        row['target'] += rules_file['ingreso']['value_09']
    elif rules_file['ingreso']['limites_08']['min'] <= ingreso < rules_file['ingreso']['limites_08']['max']:
        row['target'] += rules_file['ingreso']['value_08']
    elif rules_file['ingreso']['limites_07']['min'] <= ingreso < rules_file['ingreso']['limites_07']['max']:
        row['target'] += rules_file['ingreso']['value_07']
    elif rules_file['ingreso']['limites_06']['min'] <= ingreso < rules_file['ingreso']['limites_06']['max']:
        row['target'] += rules_file['ingreso']['value_06']
    elif rules_file['ingreso']['limites_05']['min'] <= ingreso < rules_file['ingreso']['limites_05']['max']:
        row['target'] += rules_file['ingreso']['value_05']
    elif rules_file['ingreso']['limites_04']['min'] <= ingreso < rules_file['ingreso']['limites_04']['max']:
        row['target'] += rules_file['ingreso']['value_04']
    elif rules_file['ingreso']['limites_03']['min'] <= ingreso < rules_file['ingreso']['limites_03']['max']:
        row['target'] += rules_file['ingreso']['value_03']
    elif rules_file['ingreso']['limites_02']['min'] <= ingreso < rules_file['ingreso']['limites_02']['max']:
        row['target'] += rules_file['ingreso']['value_02']
    elif rules_file['ingreso']['limites_01']['min'] < ingreso < rules_file['ingreso']['limites_01']['max']:
        row['target'] += rules_file['ingreso']['value_01']
    else:
        row['target'] += rules_file['ingreso']['value_00']

    if semanas >= rules_file['semanas']['limites_10']['min']:
        row['target'] += rules_file['semanas']['value_10']
    elif rules_file['semanas']['limites_08']['max'] > semanas >= rules_file['semanas']['limites_08']['min']:
        row['target'] += rules_file['semanas']['value_08']
    elif rules_file['semanas']['limites_06']['max'] > semanas >= rules_file['semanas']['limites_06']['min']:
        row['target'] += rules_file['semanas']['value_06']
    elif rules_file['semanas']['limites_05']['max'] > semanas > rules_file['semanas']['limites_05']['min']:
        row['target'] += rules_file['semanas']['value_05']
    else:
        row['target'] += rules_file['semanas']['value_00']

    if antiguedad > rules_file['antiguedad']['limites_10']['min']:
        row['target'] += rules_file['antiguedad']['value_10']
    elif rules_file['antiguedad']['limites_09']['min'] <= antiguedad <= rules_file['antiguedad']['limites_09']['max']:
        row['target'] += rules_file['antiguedad']['value_09']
    elif rules_file['antiguedad']['limites_08']['min'] <= antiguedad < rules_file['antiguedad']['limites_08']['max']:
        row['target'] += rules_file['antiguedad']['value_08']
    elif rules_file['antiguedad']['limites_07']['min'] <= antiguedad < rules_file['antiguedad']['limites_07']['max']:
        row['target'] += rules_file['antiguedad']['value_07']
    elif rules_file['antiguedad']['limites_06']['min'] <= antiguedad < rules_file['antiguedad']['limites_06']['max']:
        row['target'] += rules_file['antiguedad']['value_06']
    elif rules_file['antiguedad']['limites_05']['min'] <= antiguedad < rules_file['antiguedad']['limites_05']['max']:
        row['target'] += rules_file['antiguedad']['value_05']
    elif rules_file['antiguedad']['limites_04']['min'] <= antiguedad < rules_file['antiguedad']['limites_04']['max']:
        row['target'] += rules_file['antiguedad']['value_04']
    elif rules_file['antiguedad']['limites_03']['min'] <= antiguedad < rules_file['antiguedad']['limites_03']['max']:
        row['target'] += rules_file['antiguedad']['value_03']
    elif rules_file['antiguedad']['limites_02']['min'] <= antiguedad < rules_file['antiguedad']['limites_02']['max']:
        row['target'] += rules_file['antiguedad']['value_02']
    elif rules_file['antiguedad']['limites_01']['min'] <= antiguedad < rules_file['antiguedad']['limites_01']['max']:
        row['target'] += rules_file['antiguedad']['value_01']
    else:
        row['target'] += rules_file['antiguedad']['value_00']

    if tiempo_desempleado == rules_file['tiempo_desempleado']['limites_06']['min']:
        row['target'] += rules_file['tiempo_desempleado']['value_06']
        if antiguedad == 0:
            row['target'] -= rules_file['tiempo_desempleado']['value_01']
    elif rules_file['tiempo_desempleado']['limites_04']['max'] >= tiempo_desempleado > rules_file['tiempo_desempleado']['limites_04']['min']:
        row['target'] += rules_file['tiempo_desempleado']['value_04']
    elif rules_file['tiempo_desempleado']['limites_03']['max'] >= tiempo_desempleado > rules_file['tiempo_desempleado']['limites_03']['min']:
        row['target'] += rules_file['tiempo_desempleado']['value_03']
    elif rules_file['tiempo_desempleado']['limites_01']['max'] >= tiempo_desempleado > rules_file['tiempo_desempleado']['limites_01']['min']:
        row['target'] += rules_file['tiempo_desempleado']['value_01']
    else:
        row['target'] += rules_file['tiempo_desempleado']['value_00']

    if trabajos == rules_file['trabajos']['limites_10']['min']:
        row['target'] += rules_file['trabajos']['value_10']
    elif trabajos == rules_file['trabajos']['limites_08']['min']:
        row['target'] += rules_file['trabajos']['value_08']
    elif trabajos == rules_file['trabajos']['limites_06']['min']:
        row['target'] += rules_file['trabajos']['value_06']
    elif trabajos == rules_file['trabajos']['limites_05']['min']:
        row['target'] += rules_file['trabajos']['value_05']
    elif trabajos == rules_file['trabajos']['limites_04']['min']:
        row['target'] += rules_file['trabajos']['value_04']
    elif rules_file['trabajos']['limites_03']['min'] <= trabajos <= rules_file['trabajos']['limites_03']['max']:
        row['target'] += rules_file['trabajos']['value_03']
    elif trabajos == rules_file['trabajos']['limites_02']['min']:
        row['target'] += rules_file['trabajos']['value_02']
    elif trabajos == rules_file['trabajos']['limites_01']['min']:
        row['target'] += rules_file['trabajos']['value_01']
    else:
        row['target'] += rules_file['trabajos']['value_00']

    if rules_file['crecimiento']['limites_10']['min'] <= crecimiento <= rules_file['crecimiento']['limites_10']['max']:
        row['target'] += rules_file['crecimiento']['value_10']
    elif (rules_file['crecimiento']['limites_08']['min1'] <= crecimiento < rules_file['crecimiento']['limites_08']['max1']) \
        or (rules_file['crecimiento']['limites_08']['min2'] < crecimiento <= rules_file['crecimiento']['limites_08']['max2']):
        row['target'] += rules_file['crecimiento']['value_08']
    elif (rules_file['crecimiento']['limites_05']['min1'] <= crecimiento < rules_file['crecimiento']['limites_05']['max1']) \
        or (crecimiento > rules_file['crecimiento']['limites_05']['max2']):
        row['target'] += rules_file['crecimiento']['value_05']
    elif rules_file['crecimiento']['limites_04']['min'] <= crecimiento < rules_file['crecimiento']['limites_04']['max']:
        row['target'] += rules_file['crecimiento']['value_04']
    elif rules_file['crecimiento']['limites_03']['min'] <= crecimiento < rules_file['crecimiento']['limites_03']['max']:
        row['target'] += rules_file['crecimiento']['value_03']
    elif rules_file['crecimiento']['limites_02']['min'] <= crecimiento < rules_file['crecimiento']['limites_02']['max']:
        row['target'] += rules_file['crecimiento']['value_02']
    else:
        row['target'] += rules_file['crecimiento']['value_00']

    if decil == rules_file['decil']['limites_10']['min']:
        row['target'] += rules_file['decil']['value_10']
    elif decil == rules_file['decil']['limites_08']['min']:
        row['target'] += rules_file['decil']['value_08']
    elif decil == rules_file['decil']['limites_07']['min']:
        row['target'] += rules_file['decil']['value_07']
    elif decil == rules_file['decil']['limites_06']['min']:
        row['target'] += rules_file['decil']['value_06']
    elif rules_file['decil']['limites_05']['min'] <= decil <= rules_file['decil']['limites_05']['max']:
        row['target'] += rules_file['decil']['value_05']
    elif decil == rules_file['decil']['limites_03']['min']:
        row['target'] += rules_file['decil']['value_03']
    elif decil == rules_file['decil']['limites_02']['min']:
        row['target'] += rules_file['decil']['value_02']
    else:
        row['target'] += rules_file['decil']['value_00']

    if liquidez >= rules_file['liquidez']['limites_10']['min']:
        row['target'] += rules_file['liquidez']['value_10']
    elif rules_file['liquidez']['limites_09']['min'] <= liquidez < rules_file['liquidez']['limites_09']['max']:
        row['target'] += rules_file['liquidez']['value_09']
    elif rules_file['liquidez']['limites_08']['min'] <= liquidez < rules_file['liquidez']['limites_08']['max']:
        row['target'] += rules_file['liquidez']['value_08']
    elif rules_file['liquidez']['limites_07']['min'] <= liquidez < rules_file['liquidez']['limites_07']['max']:
        row['target'] += rules_file['liquidez']['value_07']
    elif rules_file['liquidez']['limites_06']['min'] <= liquidez < rules_file['liquidez']['limites_06']['max']:
        row['target'] += rules_file['liquidez']['value_06']
    elif rules_file['liquidez']['limites_05']['min'] <= liquidez < rules_file['liquidez']['limites_05']['max']:
        row['target'] += rules_file['liquidez']['value_05']
    elif rules_file['liquidez']['limites_04']['min'] <= liquidez < rules_file['liquidez']['limites_04']['max']:
        row['target'] += rules_file['liquidez']['value_04']
    elif rules_file['liquidez']['limites_03']['min'] <= liquidez < rules_file['liquidez']['limites_03']['max']:
        row['target'] += rules_file['liquidez']['value_03']
    elif rules_file['liquidez']['limites_02']['min'] <= liquidez < rules_file['liquidez']['limites_02']['max']:
        row['target'] += rules_file['liquidez']['value_02']
    elif rules_file['liquidez']['limites_01']['min'] <= liquidez < rules_file['liquidez']['limites_01']['max']:
        row['target'] += rules_file['liquidez']['value_01']
    else:
        row['target'] += rules_file['liquidez']['value_00']

    if costo >= rules_file['costo']['limites_10']['min']:
        row['target'] += rules_file['costo']['value_10']
    elif rules_file['costo']['limites_09']['min'] <= costo < rules_file['costo']['limites_09']['max']:
        row['target'] += rules_file['costo']['value_09']
    elif rules_file['costo']['limites_08']['min'] <= costo < rules_file['costo']['limites_08']['max']:
        row['target'] += rules_file['costo']['value_08']
    elif rules_file['costo']['limites_07']['min'] <= costo < rules_file['costo']['limites_07']['max']:
        row['target'] += rules_file['costo']['value_07']
    elif rules_file['costo']['limites_06']['min'] <= costo < rules_file['costo']['limites_06']['max']:
        row['target'] += rules_file['costo']['value_06']
    elif rules_file['costo']['limites_05']['min'] <= costo < rules_file['costo']['limites_05']['max']:
        row['target'] += rules_file['costo']['value_05']
    elif rules_file['costo']['limites_04']['min'] <= costo < rules_file['costo']['limites_04']['max']:
        row['target'] += rules_file['costo']['value_04']
    elif rules_file['costo']['limites_03']['min'] <= costo < rules_file['costo']['limites_03']['max']:
        row['target'] += rules_file['costo']['value_03']
    elif rules_file['costo']['limites_02']['min'] <= costo < rules_file['costo']['limites_02']['max']:
        row['target'] += rules_file['costo']['value_02']
    elif rules_file['costo']['limites_01']['min'] <= costo < rules_file['costo']['limites_01']['max']:
        row['target'] += rules_file['costo']['value_01']
    else:
        row['target'] += rules_file['costo']['value_00']

    if jubilado == 1:
        if antiguedad == 0:
            row['target'] += rules_file['jubilado']['value_05']
            row['target'] += rules_file['jubilado']['value_05']
            row['target'] += rules_file['jubilado']['value_05']
            row['target'] += rules_file['jubilado']['value_05']

    if region > rules_file['region']['limites_09']['min']:
        row['target'] += rules_file['region']['value_09']
    elif rules_file['region']['limites_08']['max'] >= region > rules_file['region']['limites_08']['min']:
        row['target'] += rules_file['region']['value_08']
    elif rules_file['region']['limites_07']['max'] >= region > rules_file['region']['limites_07']['min']:
        row['target'] += rules_file['region']['value_07']
    elif rules_file['region']['limites_06']['max'] >= region > rules_file['region']['limites_06']['min']:
        row['target'] += rules_file['region']['value_06']
    elif rules_file['region']['limites_05']['max'] >= region > rules_file['region']['limites_05']['min']:
        row['target'] += rules_file['region']['value_05']
    elif rules_file['region']['limites_04']['max'] >= region > rules_file['region']['limites_04']['min']:
        row['target'] += rules_file['region']['value_04']
    elif rules_file['region']['limites_03']['max'] >= region > rules_file['region']['limites_03']['min']:
        row['target'] += rules_file['region']['value_03']
    elif rules_file['region']['limites_02']['max'] >= region > rules_file['region']['limites_02']['min']:
        row['target'] += rules_file['region']['value_02']
    elif region < rules_file['region']['limites_01']['max']:
        row['target'] += rules_file['region']['value_01']
    else:
        row['target'] += rules_file['region']['value_00']

    return row

### Applying the create_target function to set up the target in our dataframe

In [252]:
df_scoring = df_scoring.apply(creating_target, axis=1)
df_scoring['target'] = round((df_scoring['target'] / 12) , 2)

### Rescaling the target to 300 - 850

In [253]:
def rescale(df, column, new_min, new_max):
    df_copy = df.copy()
    old_min = df_copy[column].min()
    old_max = df_copy[column].max()

    df_copy[column] = (df_copy[column] - old_min) * (new_max - new_min) / (old_max - old_min) + new_min
    return df_copy

### Applying the rescale function

In [254]:
df_scoring = rescale(df_scoring, 'target', 0.3, 0.85)

# MAKING A FINAL CSV TO BE SENT TO MACHINE LEARNING MODEL

In [255]:
df_ml = df_scoring.drop(['lugar_actual','patronActual','jubilado'] , axis= 1)

In [256]:
df_scoring.tail()

,ingreso,antiguedad_laboral_meses,tiempo_desempleado,trabajos_ultimos_5,semanasCotizadas,edad,crecimiento_ingreso,lugar_actual,patronActual,jubilado,decil_ingreso_ENIGH,liquidez_porcentual,costo_de_vida,crecimiento_gral,target
26334,11130.01,15,0,1,60,51,0.00,CHIHUAHUA,COPLASCORP,0,5.0,-28.221269,1.606810,2.576073,0.594928
26335,0.00,0,29,2,27,37,0.00,COAHUILA DE ZARAGOZA,Sin dato,0,0.0,-100.000000,-100.000000,0.694582,0.403623
26336,0.00,0,6,5,572,35,0.00,JALISCO,Sin dato,0,0.0,-100.000000,-100.000000,2.125347,0.427536
26337,39363.51,7,0,5,1097,44,83.61,QUERÉTARO,INX INTERNATIONAL DE MEXICO,0,9.0,156.925201,175.250052,2.542859,0.746377
26338,10820.06,51,0,1,428,30,0.00,GUANAJUATO,DENSO MEXICO,0,5.0,-17.423033,-2.898142,2.528201,0.658696


In [258]:
df_ml.to_csv('../dataset/data_prueba_limpia.csv', index=False)